# REAL TIME ACCIDENT DETECTION

<img src="main.jpg">

We first take a video input, then we divide it into frames, we will predict the next frame, using the current frame. A major error in actual frame and predicted frame will be considered as an anomaly. These frames are taken to a convolutional layer to predict any accident or harsh driving behaviours exhibited by any driver.


## To Find Video Anomalies We Used PREDNET Model
<img src="prednet_model.png">


## Here Is The Code

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import ConvLSTM2D,TimeDistributed,MaxPooling2D,UpSampling2D,Conv2D,Input,merge
from keras.models import Model

Using TensorFlow backend.


In [2]:
def lstm_Cnn2d(input_shape):
    c = 64
    input_img = Input(input_shape,name = 'input')
    x = ConvLSTM2D(c,(3,3),padding='same',return_sequences=True)(input_img)
    x = ConvLSTM2D(c,(3,3),padding='same',return_sequences=True)(x)
    c1 = ConvLSTM2D(c,(3,3),padding='same',return_sequences=True)(x) 
    
    x = TimeDistributed(MaxPooling2D((2,2),(2,2)))(c1)
    
    x = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    x = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    c2 = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    
    x = TimeDistributed(MaxPooling2D((2,2),(2,2)))(c2)
    x = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    x = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    c3 = ConvLSTM2D(2*c,(3,3),padding='same',return_sequences=True)(x)
    
    x = TimeDistributed(UpSampling2D((2,2)))(c3)
    x = merge.concatenate([c2,x])
    x = TimeDistributed(Conv2D(c,(3,3),padding='same'))(x)
    
    x = TimeDistributed(UpSampling2D((2,2)))(x)
    x = merge.concatenate([c1,x])

    x = TimeDistributed(Conv2D(3,(3,3),padding='same'))(x)
    x = TimeDistributed(UpSampling2D((2,2)))(x)
    x = TimeDistributed(Conv2D(3,(3,3),padding='same'))(x)
    x = TimeDistributed(UpSampling2D((2,2)))(x)
    
    output = TimeDistributed(Conv2D(3,(3,3),padding='same',activation='softmax'),name='output')(x)
    
    model = Model(input_img,output=[output])
    model.compile(loss='categorical_crossentropy',optimizer = 'adadelta')
    return model

In [3]:
batch_size = 10 
pixel_x = 64
pixel_y = 64
color_code = 3
Model = lstm_Cnn2d((batch_size,pixel_x,pixel_y,color_code))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=[<tf.Tenso...)`


In [4]:
Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 10, 64, 64, 3 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)     (None, 10, 64, 64, 6 154624      input[0][0]                      
__________________________________________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)     (None, 10, 64, 64, 6 295168      conv_lst_m2d_1[0][0]             
__________________________________________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)     (None, 10, 64, 64, 6 295168      conv_lst_m2d_2[0][0]             
__________________________________________________________________________________________________
time_distr

## Resources And Research

PredNet :- https://coxlab.github.io/prednet/